In [63]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
print("Imported!")

Imported!


In [64]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
 
wikidata = BeautifulSoup(source, 'html5lib')

In [65]:
table = {} # initialize an empty dictionary to save the data in

for table_cell in wikidata.find_all('td'):
   try:
       postal_code = table_cell.p.b.text # get the postal code
       postal_code_i = table_cell.span.text
       neighborhoods_data = table_cell.span.text # get the rest of the data in the cell
       borough = neighborhoods_data.split('(')[0] # get the borough in the cell
      
       # if the cell is not assigned then ignore it
       if neighborhoods_data == 'Not assigned':
           neighborhoods = []
       # else process the data and add it to the dictionary
       else:
           table[postal_code] = {}
          
           try:
               neighborhoods = neighborhoods_data.split('(')[1]
          
               # remove parantheses from neighborhoods string
               neighborhoods = neighborhoods.replace('(', ' ')
               neighborhoods = neighborhoods.replace(')', ' ')
 
               neighborhoods_names = neighborhoods.split('/')
               neighborhoods_clean = ', '.join([name.strip() for name in neighborhoods_names])
           except:
               borough = borough.strip('\n')
               neighborhoods_clean = borough
           # add borough and neighborhood to dictionary
           table[postal_code]['borough'] = borough
           table[postal_code]['neighborhoods'] = neighborhoods_clean
   except:
       pass
table

{'M3A': {'borough': 'North York', 'neighborhoods': 'Parkwoods'},
 'M4A': {'borough': 'North York', 'neighborhoods': 'Victoria Village'},
 'M5A': {'borough': 'Downtown Toronto',
  'neighborhoods': 'Regent Park, Harbourfront'},
 'M6A': {'borough': 'North York',
  'neighborhoods': 'Lawrence Manor, Lawrence Heights'},
 'M7A': {'borough': "Queen's Park",
  'neighborhoods': 'Ontario Provincial Government'},
 'M9A': {'borough': 'Etobicoke', 'neighborhoods': 'Islington Avenue'},
 'M1B': {'borough': 'Scarborough', 'neighborhoods': 'Malvern, Rouge'},
 'M3B': {'borough': 'North York', 'neighborhoods': 'Don Mills North'},
 'M4B': {'borough': 'East York',
  'neighborhoods': 'Parkview Hill, Woodbine Gardens'},
 'M5B': {'borough': 'Downtown Toronto',
  'neighborhoods': 'Garden District, Ryerson'},
 'M6B': {'borough': 'North York', 'neighborhoods': 'Glencairn'},
 'M9B': {'borough': 'Etobicoke',
  'neighborhoods': 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'},
 'M1C': {'boro

In [66]:
column_names = ['PostalCode','Borough','Neighborhoods']
toronto = pd.DataFrame(columns = column_names)

# populate dataframe with data from dictionary
for ind, postal_code in enumerate(table):
   borough = table[postal_code]['borough']
   neighborhood = table[postal_code]['neighborhoods']
   toronto = toronto.append({"PostalCode": postal_code,
                                       "Borough": borough,
                                       "Neighborhoods": neighborhood},
                                       ignore_index=True)

toronto

,PostalCode,Borough,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [67]:
toronto.shape

(103, 3)

In [68]:
# loading geographical data
geo_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
geo_df = pd.read_csv(geo_url)
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [69]:
# joining geographical data
toronto_neigh=toronto.merge(geo_df, left_on='PostalCode',right_on='Postal Code',how='outer').drop(columns='Postal Code')
toronto_neigh.head()

,PostalCode,Borough,Neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [70]:
toronto_neigh.shape

(103, 5)

In [71]:
#Segmenting and Clustering Neighborhoods in Toronto

In [72]:
!pip install geocoder
!pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [73]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("All Required Libraries Imported!")

All Required Libraries Imported!


In [74]:
# get the latitude and longitude values of Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [75]:
# filter data to include only boroughs that contain the word Toronto
toronto_data = toronto_neigh.loc[toronto_neigh['Borough'].str.contains('Toronto')].copy()
print(toronto_data.shape)
toronto_data.head()

(39, 5)


,PostalCode,Borough,Neighborhoods,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [76]:
# creating the map of Toronto with markers of locations
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhoods']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [77]:
CLIENT_ID = 'GWLMNRDZRSIHGZYE2HIBOKZB4IMUEBQD0XM1VQQ1HDT42TRD'
CLIENT_SECRET = 'FQWBEZ1IAJH1IN420UPDUKO0SELXVNRYCQ5EC4DH0OB4ETPW'
VERSION = '20180605'
LIMIT = 60



In [78]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [79]:
# explode dataframe to search each nieghborhood in list of neighborhoods
toronto_data['Neighborhoods_list'] = toronto_data['Neighborhoods'].apply(lambda x: x.split(', '))
toronto_ex = toronto_data.explode('Neighborhoods_list').reset_index(drop=True).rename(columns={'Neighborhoods_list':'Neighborhood'})
toronto_ex.head(10)

,PostalCode,Borough,Neighborhoods,Latitude,Longitude,Neighborhood
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Regent Park
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Harbourfront
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Garden District
3,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Ryerson
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,St. James Town
5,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Beaches
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,Berczy Park
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,Central Bay Street
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564,Christie
9,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,Richmond


In [80]:
toronto_venues = getNearbyVenues(names=toronto_ex['Neighborhood'],
                                   latitudes=toronto_ex['Latitude'],
                                   longitudes=toronto_ex['Longitude']
                                  )

Regent Park
Harbourfront
Garden District
Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond
Adelaide
King
Dufferin
Dovercourt Village
The Danforth  East
Harbourfront East
Union Station
Toronto Islands
Little Portugal
Trinity
The Danforth West
Riverdale
Toronto Dominion Centre
Design Exchange
Brockton
Parkdale Village
Exhibition Place
India Bazaar
The Beaches West
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
University of Toronto
Harbord
Runnymede
Swansea
Moore Park
Summerhill East
Kensington Market
Chinatown
Grange Park
Summerhill West
Rathnelly
South Hill
Forest Hill SE
Deer Park
CN Tower
King and Spadina
Railway Lands
Harbourfront West
Bathurst Quay
South Niagara
Island airport
Rosedale
Enclave of M5E
St. James Town
Cabbagetown
First Canadian Place
Underground city
Church a

In [81]:
print(f"""{toronto_venues.shape[0]} venues in {len(toronto_venues['Venue Category'].unique())} venue categories""")

2414 venues in 223 venue categories


In [82]:
# explore venues by neighborhood
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Adelaide                     60
Bathurst Quay                16
Berczy Park                  60
Brockton                     23
CN Tower                     16
Cabbagetown                  44
Central Bay Street           60
Chinatown                    60
Christie                     16
Church and Wellesley         60
Commerce Court               60
Davisville                   34
Davisville North             12
Deer Park                    14
Design Exchange              60
Dovercourt Village           14
Dufferin                     14
Enclave of M4L               15
Enclave of M5E               60
Exhibition Place             23
First Canadian Place         60
Forest Hill North & West      4
Forest Hill SE               14
Garden District              60
Grange Park                  60
Harbord                      32
Harbourfront                 45
Harbourfront East            60
Harbourfront West            16
High Park                    25
India Bazaar               

In [83]:
# from above we can see the robust neighborhoods are: Adelaide, Berczy Park, Central Bay Street, Chinatown,Church and Wellesley,
# Commerce Court, Design Exchange,Enclave of M5E, First Canadian Place, Garden District, Grange Park, Harbourfront East, 
# Kensington Market, King,Richmond, Ryerson, St. James Town,Toronto Dominion Centre,Toronto Islands, Underground city, Union Station, Victoria Hotel


In [84]:
#Explore the common venues across the neighborhoods
toronto_venues['Venue Category'].value_counts()

Coffee Shop                        191
Café                               155
Restaurant                          70
Park                                55
Hotel                               50
Pizza Place                         49
Bakery                              47
Italian Restaurant                  45
Japanese Restaurant                 44
Sandwich Place                      37
Pub                                 37
American Restaurant                 33
Sushi Restaurant                    32
Seafood Restaurant                  32
Vegetarian / Vegan Restaurant       31
Bar                                 30
Gym                                 29
Beer Bar                            28
Gastropub                           28
Bookstore                           26
Grocery Store                       24
Brewery                             23
Greek Restaurant                    22
Theater                             22
Steakhouse                          21
Bank                     

In [85]:
#observation: People in Toronto must love coffee, with so many coffee shops and Cafe in the city.

In [86]:
# one hot encoding to get dummy values for categories
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hospital,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park,0,0,0,

In [87]:
toronto_onehot.shape

(2414, 223)

In [88]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hospital,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Adelaide,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.033333,0.0,0.0,0.000000,0.016667,0.0,0.016667,0.0,0.0,0.0,0.0,0.000000,0.016667,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.016667,0.0000,0.016667,0.000000,0.0,0.0,0.0,0.016667,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.100000,0.0,0.0,0.0,0.016667,0.000000,0.0,0.033333,0.000000,0.033333,0.0,0.000000,0.0,0.0,0.0,0.016667,0.016667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.016667,0.0,0.0,0.0,0.0,0.000000,0.016667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0.0,0.016667,0.000000,0.000000,0.000000,0.016667,0.016667,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.000000,0.0,

In [89]:
toronto_grouped.shape

(73, 223)

In [90]:
#find out the top 5 venues in each neiborhood by getting the frequency of the venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
          venue  freq
0   Coffee Shop  0.10
1          Café  0.07
2    Restaurant  0.03
3   Salad Place  0.03
4  Concert Hall  0.03


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4     Boat or Ferry  0.06


----Berczy Park----
          venue  freq
0   Coffee Shop  0.10
1  Cocktail Bar  0.07
2        Bakery  0.05
3    Restaurant  0.03
4      Beer Bar  0.03


----Brockton----
                    venue  freq
0                    Café  0.13
1             Coffee Shop  0.09
2          Breakfast Spot  0.09
3                     Gym  0.04
4  Furniture / Home Store  0.04


----CN Tower----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4     Boat or Ferry  0.06


----Cabbagetown----
                venue  freq
0              Bakery  0.07
1         Coffee Shop  0.07
2          Restaurant  0.

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neigh_venues_sorted = pd.DataFrame(columns=columns)
neigh_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neigh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neigh_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Concert Hall,Cosmetics Shop,Steakhouse,Restaurant,Pizza Place,Thai Restaurant,American Restaurant,Sushi Restaurant
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Coffee Shop
2,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Pharmacy,Restaurant,Seafood Restaurant,Beer Bar,Sandwich Place
3,Brockton,Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bakery,Pet Store,Convenience Store,Climbing Gym,Burrito Place,Restaurant
4,CN Tower,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Coffee Shop


Observations to inform hypothesis of potential types of neighborhoods and sample neighborhood names:
1) Neighborhoods for people to hang out: Adelaide,  Brockton, Berczy Park;
2) Neighborhoods to explore more culture and foods: Chinatown, Church and Wellesley, First Canadian Place;
3) Neighborhoods for commute: Bathurst Quary, CN Tower;
4) Neighborhoods residential: Christie, Deer Park, Dufferin

In [98]:
#Clustering Neighborhoods
# set number of clusters
kclusters = 5

toronto_grouped_c = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_c)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 0, 3,
       2, 2, 2, 2, 4, 2, 2, 2, 3, 0, 2, 2, 1, 3, 2, 2, 2, 2, 4, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [99]:
# add clustering labels
neigh_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_ex

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neigh_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,PostalCode,Borough,Neighborhoods,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Regent Park,2,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Wine Shop,Electronics Store,Performing Arts Venue
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Harbourfront,2,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Wine Shop,Electronics Store,Performing Arts Venue
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Garden District,2,Clothing Store,Cosmetics Shop,Coffee Shop,Café,Ramen Restaurant,Bookstore,Middle Eastern Restaurant,Burger Joint,Pizza Place,Theater
3,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Ryerson,2,Clothing Store,Cosmetics Shop,Coffee Shop,Café,Ramen Restaurant,Bookstore,Middle Eastern Restaurant,Burger Joint,Pizza Place,Theater
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,St. James Town,2,Café,Coffee Shop,Restaurant,Bakery,Park,Gastropub,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Diner


In [100]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, bor,poi, cluster in zip(
    toronto_merged['Latitude'], 
    toronto_merged['Longitude'], 
    toronto_merged['Borough'],
    toronto_merged['Neighborhood'], 
    toronto_merged['Cluster Labels']
):
    label = folium.Popup(str(bor) + ' | ' + str(poi) + ' | Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [101]:
# number of boroughs and neighborhoods in each cluster

print('Cluster','\tBoroughs','Neighborhoods',sep='\t')
print('-'*46)
clusters = toronto_merged.groupby('Cluster Labels')
for name,data in clusters:
    print(name,data['Borough'].nunique(),data['Neighborhood'].nunique(),sep='\t\t')

Cluster		Boroughs	Neighborhoods
----------------------------------------------
0		1		7
1		1		2
2		6		57
3		1		5
4		2		2


Cluster observation

Cluster 1: Commute Centers: where offers access to airports, ferrys, and a few facilities like shopping boutique, food and coffee shops for passengers waiting for their commute.

In [105]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,CN Tower,0,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Coffee Shop
60,Downtown Toronto,King and Spadina,0,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Coffee Shop
61,Downtown Toronto,Railway Lands,0,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Coffee Shop
62,Downtown Toronto,Harbourfront West,0,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Coffee Shop
63,Downtown Toronto,Bathurst Quay,0,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Coffee Shop
64,Downtown Toronto,South Niagara,0,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Coffee Shop
65,Downtown Toronto,Island airport,0,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Coffee Shop


Cluster 2: Outdoor lounge / entertainment space: here people can take their chidren to the playground or have a match at the tennis courts, and afterwards relax with wine or at a dive bar, or have a quick bite at the dumpling restaurents

In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Central Toronto,Moore Park,1,Playground,Tennis Court,Wine Shop,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Distribution Center
50,Central Toronto,Summerhill East,1,Playground,Tennis Court,Wine Shop,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Distribution Center


Cluster 3: Commercial centers: lots of choice of places to hang out, explore different food and culture, socialize, or see a show.

In [107]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Regent Park,2,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Wine Shop,Electronics Store,Performing Arts Venue
1,Downtown Toronto,Harbourfront,2,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Wine Shop,Electronics Store,Performing Arts Venue
2,Downtown Toronto,Garden District,2,Clothing Store,Cosmetics Shop,Coffee Shop,Café,Ramen Restaurant,Bookstore,Middle Eastern Restaurant,Burger Joint,Pizza Place,Theater
3,Downtown Toronto,Ryerson,2,Clothing Store,Cosmetics Shop,Coffee Shop,Café,Ramen Restaurant,Bookstore,Middle Eastern Restaurant,Burger Joint,Pizza Place,Theater
4,Downtown Toronto,St. James Town,2,Café,Coffee Shop,Restaurant,Bakery,Park,Gastropub,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Diner
5,East Toronto,The Beaches,2,Asian Restaurant,Pub,Trail,Health Food Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Wine Shop
6,Downtown Toronto,Berczy Park,2,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Pharmacy,Restaurant,Seafood Restaurant,Beer Bar,Sandwich Place
7,Downtown Toronto,Central Bay Street,2,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Portuguese Restaurant,Poke Place,Miscellaneous Shop
8,Downtown Toronto,Christie,2,Grocery Store,Café,Park,Coffee Shop,Candy Store,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub
9,Downtown Toronto,Richmond,2,Coffee Shop,Café,Concert Hall,Cosmetics Shop,Steakhouse,Restaurant,Pizza Place,Thai Restaurant,American Restaurant,Sushi Restaurant


Cluster 4: Multi-cultural Neighborhood: easy access to different kinds of restaurants and to get everyday errands done.

In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Central Toronto,Summerhill West,3,Coffee Shop,Restaurant,Sushi Restaurant,Bagel Shop,Bank,Fried Chicken Joint,Pub,Pizza Place,American Restaurant,Liquor Store
55,Central Toronto,Rathnelly,3,Coffee Shop,Restaurant,Sushi Restaurant,Bagel Shop,Bank,Fried Chicken Joint,Pub,Pizza Place,American Restaurant,Liquor Store
56,Central Toronto,South Hill,3,Coffee Shop,Restaurant,Sushi Restaurant,Bagel Shop,Bank,Fried Chicken Joint,Pub,Pizza Place,American Restaurant,Liquor Store
57,Central Toronto,Forest Hill SE,3,Coffee Shop,Restaurant,Sushi Restaurant,Bagel Shop,Bank,Fried Chicken Joint,Pub,Pizza Place,American Restaurant,Liquor Store
58,Central Toronto,Deer Park,3,Coffee Shop,Restaurant,Sushi Restaurant,Bagel Shop,Bank,Fried Chicken Joint,Pub,Pizza Place,American Restaurant,Liquor Store


Cluster 5: Urban Residential: offers green spaces like park and playground and also easy access to groceries, restaurants, and post offices

In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,East YorkEast Toronto,The Danforth East,4,Park,Coffee Shop,Convenience Store,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Distribution Center
66,Downtown Toronto,Rosedale,4,Park,Playground,Trail,Wine Shop,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Distribution Center
